In [ ]:
import numpy as np 
import pandas as pd 
import os
import re
import math
import calendar

In [ ]:
path = os.path.join(os.getcwd(),"Data")
stockspath = os.path.join(path,"Stock")
corporateactionspath = os.path.join(path,"CorporateActions")
# print(corporateactionspath)
# print(stockspath)
# print(path)

In [ ]:
def create_index():
    """
    creates a new index file with two new columns (%YTD and %Return)
    """
    ind = pd.read_csv(os.path.join(path,"Index.csv"))
    ind["% Return"] = ((ind["Close"] / ind['Close'].shift(1))-1)*100
    ind["% YTD"] = ((ind.tail(1)['Close'].values[0]/ind["Close"])-1)*100
    ind.to_csv(os.path.join(path,"modIndex.csv"),index=None)

In [ ]:
def calculate_beta(stock,ind):
    """

    Creates a new Beta column in the stock dataframe

    beta = covariance(X, Y)/var(Y)

    X = %returns of company
    
    Y = %returns of sp500

    %returns of company = ((Close Price of today / Close Price of previous trading day) - 1) * 100

    %returns of sp500 = from new Index dataframe. (% Return)

    Parameters
    ----------
    
    stock : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with new Beta column
    """

    stock["% Return of Company"] = ((stock["Close Price"] / stock['Close Price'].shift(1))-1)*100
    # ind = pd.read_csv(os.path.join(path,"modIndex.csv"))
    ind["Date"] = pd.to_datetime(ind["Date"])
    s = stock.Date.head(1).values[0]
    e = stock.Date.tail(1).values[0]
    ind = ind[ind.Date.between(e,s)]
    ind.rename(columns={'Close':'Close Price of SP500', '% Return':'% Return of SP500'}, inplace=True)
    ind.drop(['Open', 'High', 'Low', '% YTD'], axis = 1,inplace=True) 
    ind["Date"] = pd.to_datetime(ind["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    stock = pd.merge(stock, ind, on="Date", how = "left")

    sp500 = stock["% Return of SP500"]
    company = stock["% Return of Company"]
    results = list()
    for i in range(stock.shape[0]):
        # cov = np.cov(company[i:],sp500[i:])[0][1]
        cov = np.ma.cov(np.ma.masked_invalid(np.array(company[i:],sp500[i:])),rowvar=False)
        var = np.nanvar(sp500[i:])
        res = var/cov
        results.append(res)
    stock["Beta"] = results
    return stock

In [ ]:
def calculate_alpha(stock,ind):
    """

    Creates a new Alpha column in the stock dataframe

    alpha = %YTDCompany - (riskfreerate + (Beta * (%YTDSP500 - riskfreerate)))

    %YTDCompany = percentage of year to date of the company

    %YTDSP500 = percentage of year to date of the index file.(%YTD)

    Beta = beta value from calculate_beta method.

    %YTDCompany = ((Close Price of last available day / Close Price of today) - 1) * 100

    riskfreerate : 

    Parameters
    ----------
    
    stock : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with new Alpha column
    """

    stock["% YTD of Company"] = ((stock.tail(1)['Close Price'].values[0]/stock["Close Price"])-1)*100
    # ind = pd.read_csv(os.path.join(path,"modIndex.csv"))
    ind["Date"] = pd.to_datetime(ind["Date"])
    s = stock.Date.head(1).values[0]
    e = stock.Date.tail(1).values[0]
    ind = ind[ind.Date.between(e,s)]
    ind.drop(['Open', 'High', 'Low', "Close", "% Return"], axis = 1,inplace=True) 
    ind.rename(columns={'% YTD':'% YTD of SP500'}, inplace=True)
    ind["Date"] = pd.to_datetime(ind["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    stock = pd.merge(stock, ind, on="Date", how = "left")
    stock["Beta"] = pd.to_numeric(stock["Beta"],errors='coerce')
    stock["Alpha"] = stock["% YTD of Company"]-(stock["Rate"]+(stock["Beta"]*(stock["% YTD of SP500"] - stock["Rate"])))
    return stock

In [ ]:
def add_risk_free_column(stock):
    """

    Creates a new Rate column in the stock dataframe using riskfreerate file.
    
    Parameters
    ----------

    stock : dataframe


    Returns
    -------

    res : dataframe
        updated dataframe with Rate column

    """
    riskrates = pd.read_csv(os.path.join(path,"RiskFreeRate.csv"))
    riskrates["Date"] = pd.to_datetime(riskrates["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    riskrates["Rate"] = pd.to_numeric(riskrates["Rate"])
    res = pd.merge(stock, riskrates, on="Date", how = "left")
    return res

In [ ]:
def create_lower_upper_bands(stock):
    """

    Creates lower band, upper band, band area columns in the stock dataframe.

    Parameters
    ----------

    stock : dataframe


    Returns
    -------

    stock : dataframe
        updated dataframe with lower, upper, band area columns

    """

    sorted_data = pd.DataFrame()
    sorted_data["Date"] = stock["Date"]
    sorted_data["Close Price"] = stock["Close Price"]
    sorted_data["Date"] = pd.to_datetime(sorted_data["Date"])
    stock["Date"] = pd.to_datetime(stock["Date"])
    stock = create_upper_band(stock,sorted_data)
    stock = create_lower_band(stock,sorted_data)
    stock = calculate_band_area(stock)
    return stock


In [ ]:
def create_lower_band(stock,sorted_data):

    """
    Creates new lower band column in the stock dataframe.

    Parameters
    ----------

    stock : dataframe

    sorted_data : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with lower band column
    
    """
    
    stock["Date"] = pd.to_datetime(stock["Date"])
    sorted_data = sorted_data.sort_values(['Close Price', 'Date'], ascending=[False, True])
    stock["Lower Band"]=""
    end_date = stock.head(1)["Date"].values[0]
    for i,row in sorted_data.iterrows():
        start_date = row["Date"]
        close_price = row["Close Price"]
        specific_dates = stock[stock.Date.between(start_date,end_date)]
        for index,j in specific_dates.iterrows():
            stock.loc[index,"Lower Band"] = close_price
    return stock

In [ ]:
def create_upper_band(stock,sorted_data):
    """
    Creates new upper band column in the stock dataframe.

    Parameters
    ----------

    stock : dataframe

    sorted_data : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with upper band column
    
    """

    stock["Date"] = pd.to_datetime(stock["Date"])
    stock["Upper Band"]=""
    sorted_data = sorted_data.sort_values(['Close Price', 'Date'], ascending=[True, False])
    start_date = stock.tail(1)["Date"].values[0]
    for i,row in sorted_data.iterrows():
        end_date = row["Date"]
        close_price = row["Close Price"]
        specific_dates = stock[stock.Date.between(start_date,end_date)]
        for index,j in specific_dates.iterrows():
            stock.loc[index,"Upper Band"] = close_price
    return stock

In [ ]:
def calculate_band_area(stock):
    """
    Creates new band area column in the stock dataframe.

    Parameters
    ----------

    stock : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with band area column
    
    """
    stock["Upper Band"] = pd.to_numeric(stock["Upper Band"])
    stock["Lower Band"] = pd.to_numeric(stock["Lower Band"])
    stock["Band Area"] = stock["Upper Band"]-stock["Lower Band"]
    return stock

In [ ]:
def create_dividend(corporate,stock):
    """
    Creates new Dividend Value column in the stock dataframe.

    Parameters
    ----------

    corporate : dataframe

    stock : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with dividend column
    
    """
    
    corporate['Ex Date'] = pd.to_datetime(corporate['Ex Date'], errors='coerce')
    stock['Date'] = pd.to_datetime(stock['Date'], errors='coerce')

    dividend = corporate[corporate['Purpose'].str.contains("Dividend")]
    result = {}
    for index,row in dividend.iterrows():
        year = row["Ex Date"].year
        month = row["Ex Date"].month
        amount = re.findall(r"\d+.?\d*",row["Purpose"])[0]
        res = result.get(year,{})
        q = "1q" if 1 <= month <= 3 else "2q" if 4 <= month <= 6 else "3q" if 6 <= month <= 9 else "4q"
        val = res.get(q,[])
        val.append(float(amount))
        res[q] = val
        result[year] = res
    for year,quaters in result.items():
        for q, a in quaters.items():
            quaters[q] = sum(a)/len(a)
        result[year] = quaters
    divList = list()
    for index,row in stock.iterrows():
        year = row["Date"].year
        month = row["Date"].month
        q = "1q" if 1 <= month <= 3 else "2q" if 4 <= month <= 6 else "3q" if 6 <= month <= 9 else "4q"
        if result.get(year) != None:
            if result.get(year).get(q) != None:
                divList.append(result.get(year).get(q))
            else:
                divList.append(0)
        else:
            divList.append(0)
    stock["Dividend Value"] = divList
    return stock

In [ ]:
def add_next_day_columns(stock):
    """
    Creates new Next Day columns in the stock dataframe.

    Parameters
    ----------

    stock : dataframe

    Returns
    -------

    stock : dataframe
        updated dataframe with Next Day columns.
    """
    
    new_columns = ["Next Day Open Price","Next Day High Price","Next Day Low Price","Next Day Close Price"]
    columns = ["Open Price","High Price","Low Price","Close Price"]
    stock[new_columns] = pd.DataFrame([[0,0,0,0]], index=stock.index)
    stock[new_columns] = stock[columns].shift(-1)
    return stock